In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
df = pd.read_excel('pm25.xlsx')
df = df.drop(columns=["No."])

df.iloc[:, 3:] = df.iloc[:, 3:].apply(lambda col: col.fillna(col.mean()))

df_pivot = df.melt(id_vars=['วันที่', 'ช่วงเวลา'], var_name='Station', value_name='Value')
df_pivot['DateTime'] = pd.to_datetime(df_pivot['วันที่'].astype(str) + ' ' + df_pivot['ช่วงเวลา'].astype(str))
df_pivot = df_pivot.pivot(index='Station', columns='DateTime', values='Value')

df_pivot = df_pivot.groupby(df_pivot.columns.date, axis=1).mean()

scaler = MinMaxScaler()
df_pivot.iloc[:, :] = scaler.fit_transform(df_pivot)
df_pivot

C:\Users\user\AppData\Local\Temp\ipykernel_11336\3116661590.py:12: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.



,2025-03-04,2025-03-05,2025-03-06,2025-03-07,2025-03-08,2025-03-09,2025-03-10,2025-03-11,2025-03-12,2025-03-13,2025-03-14,2025-03-15,2025-03-16,2025-03-17,2025-03-18,2025-03-19,2025-03-20
Station,,,,,,,,,,,,,,,,,
02t,0.504325,0.470532,0.188617,0.190538,0.170819,0.215017,0.048848,0.248891,0.354614,0.270781,0.187125,0.441219,0.268762,0.154770,0.461362,0.526412,0.422617
03t,0.520577,0.496913,0.180661,0.183430,0.160786,0.178943,0.057040,0.268351,0.388276,0.304670,0.176028,0.426250,0.246722,0.150100,0.461186,0.441348,0.360511
08t,0.628047,0.552038,0.235702,0.258109,0.267224,0.219883,0.069952,0.378241,0.512865,0.422526,0.240123,0.476269,0.235562,0.139426,0.428622,0.566302,0.422617
11t,0.414417,0.502819,0.153796,0.172808,0.113378,0.105621,0.003289,0.209113,0.328690,0.261339,0.169796,0.409456,0.215940,0.139360,0.380743,0.324044,0.236964
12t,0.415990,0.498488,0.149771,0.170439,0.110786,0.140017,0.011277,0.216649,0.328497,0.258304,0.170556,0.413107,0.213522,0.141828,0.371590,0.320947,0.238791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bkp98t,0.322936,0.441394,1.000000,1.000000,0.107776,0.149916,0.048725,0.341325,0.310698,0.331479,0.210762,0.509127,0.209709,0.193929,0.482309,0.430055,0.415975
bkp99t,0.087287,0.266175,0.033324,0.115369,0.063796,0.024497,0.020573,0.175344,0.185916,0.179228,0.165304,0.418293,0.253270,0.170254,0.376518,0.229690,0.164231
o10,0.390826,0.473682,0.131049,0.144783,0.090552,0.064430,0.016977,0.182669,0.244341,0.212780,0.130425,0.359073,0.182275,0.131021,0.353283,0.285428,0.223016


In [136]:
print(df_pivot.columns)

Index([2025-03-04, 2025-03-05, 2025-03-06, 2025-03-07, 2025-03-08, 2025-03-09,
       2025-03-10, 2025-03-11, 2025-03-12, 2025-03-13, 2025-03-14, 2025-03-15,
       2025-03-16, 2025-03-17, 2025-03-18, 2025-03-19, 2025-03-20],
      dtype='object')


In [137]:
from sklearn.cluster import KMeans
import plotly.express as px

errors=[]
for k in range(2,50):
  km=KMeans(n_clusters=k)
  km.fit(df_pivot)
  errors.append(km.inertia_)
px.line(x=range(2,50),y=errors)

In [139]:
k=best_k
km=KMeans(n_clusters=k)
km.fit(df_pivot)

KMeans(n_clusters=np.int64(3))

In [140]:
from plotly import graph_objects as go
fig=go.Figure()
for i in range(k):
  fig.add_trace(go.Scatter(x=df_pivot.columns,y=km.cluster_centers_[i],mode='lines',name=f'Cluster {i}'))
fig.show()

In [141]:
df_pivot['cluster']=km.labels_
for c in df_pivot['cluster'].unique():
  print(f'Cluster {c}')
  print(' '.join(df_pivot[df_pivot['cluster'] == c].index.astype(str)))

Cluster 2
02t 03t 11t 12t 19t 50t 52t 53t 61t 81t bkp116t bkp118t bkp120t bkp124t bkp129t bkp130t bkp57t bkp60t bkp61t bkp70t bkp71t bkp73t bkp74t bkp75t bkp77t bkp79t bkp83t bkp85t bkp86t bkp87t bkp88t bkp89t bkp90t bkp91t bkp92t bkp94t bkp96t bkp97t o10 o63
Cluster 0
08t 14t 18t 20t 54t bkp100t bkp101t bkp103t bkp105t bkp125t bkp58t bkp63t bkp64t bkp66t bkp67t bkp72t bkp78t bkp80t bkp81t bkp82t bkp84t bkp95t bkp98t
Cluster 1
13t 16t 17t 22t 27t 59t bkp102t bkp104t bkp112t bkp114t bkp117t bkp119t bkp121t bkp122t bkp123t bkp126t bkp127t bkp128t bkp131t bkp133t bkp56t bkp59t bkp62t bkp65t bkp69t bkp76t bkp93t bkp99t o26


สามารถแบ่งออกได้เป็น 3 โซน
โดยแต่ละโซนอาจจะแบ่งโดยเป็นพื้นที่ต่าง ๆ ในกรุงเทพ